Make that we are running the correct version of TensorFlow first

In [1]:
import tensorflow as tf
tf.__version__

'2.0.0-alpha0'

In [2]:
import sys

assert sys.version_info >= (3, 6) # Python ≥3.6 required
assert tf.__version__ >= "2.0"    # TensorFlow ≥2.0 required

# Train basic model on the generated emnist-lines dataset

In [65]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sys
# To be able to write code as if this notebook was one level up in files tree structure.
sys.path.append('..')

from pathlib import Path

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Dense, Input, Lambda, Reshape, TimeDistributed
from tensorflow.keras.models import Model as KerasModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import Sequence

import pandas as pd
from PIL import Image

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from recognizer.datasets import Dataset
from recognizer.datasets import EmnistDataset
from recognizer.networks import lenet
from recognizer.networks import simple
from recognizer.networks import NetworkInput

Hyperparameters

In [5]:
batch_size = 256
epochs = 4  # 32

In [83]:
model_save_path = Path("../recognizer/weights/lines_time_distributed_model.h5")

## Load characters mapping

In [59]:
import json

with open(Dataset.raw_data_path()/"emnist"/"emnist_essentials.json") as json_file:  
    mapping = json.load(json_file)["mapping"]
    mapping = {m[0]: m[1] for m in mapping}
    mapping[62] = ' '
    mapping[63] = '_'
    
mapping_reversed = {v: k for k, v in mapping.items()}

## Build the new model

In [8]:
char_image_height = 28
char_image_width = 28
max_length = 34
num_classes = 64
image_height = char_image_height
image_width = char_image_width * max_length

input_shape = (image_height, image_width)
output_shape = (max_length, num_classes)

window_width: float = 16
window_stride: float = 8
    
num_windows = ((image_width - window_width) // window_stride) + 1
if num_windows < output_length:
    raise ValueError(f'Window width/stride need to generate >= {output_length} windows (currently {num_windows})')

In [ ]:
def slide_window(image, window_width, window_stride):
    """
    Takes (image_height, image_width, 1) input,
    Returns (num_windows, image_height, window_width, 1) output, where
    num_windows is floor((image_width - window_width) / window_stride) + 1
    """
    patches = tf.image.extract_image_patches(image, 
                                             sizes=[1, 1, window_width, 1], 
                                             strides=[1, 1, window_stride, 1], 
                                             rates=[1, 1, 1, 1], 
                                             padding='VALID')
    patches = tf.transpose(patches, (0, 2, 1, 3))
    patches = tf.expand_dims(patches, -1)
    
    return patches

In [39]:
window_width = 28
window_stride = 14

image_input = Input(shape=input_shape, name='image')
image_reshaped = Reshape((image_height, image_width, 1))(image_input)

image_patches = Lambda(
    slide_window,
    arguments={'window_width': window_width, 'window_stride': window_stride}
)(image_reshaped)

In [40]:
image_patches

<tf.Tensor 'lambda_14/ExpandDims:0' shape=(None, 67, 28, 28, 1) dtype=float32>

### The model

In [52]:
convnet = lenet(NetworkInput(input_shape=(image_height, window_width, 1), number_of_classes=num_classes))
convnet.layers

In [102]:
# Get rid of the last two layers (dropout and softmax)
convnet = KerasModel(inputs=convnet.inputs, outputs=convnet.layers[-2].output)

time_distributed_outputs = TimeDistributed(convnet)(image_patches)
softmax_output = Dense(num_classes, activation='softmax', name='softmax_output')(time_distributed_outputs)

model = KerasModel(
        inputs=image_input,
        outputs=softmax_output
    )

In [103]:
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           [(None, 28, 952)]         0         
_________________________________________________________________
reshape_10 (Reshape)         (None, 28, 952, 1)        0         
_________________________________________________________________
lambda_14 (Lambda)           (None, 67, 28, 28, 1)     0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 67, 9216)          18816     
_________________________________________________________________
softmax_output (Dense)       (None, 67, 64)            589888    
Total params: 608,704
Trainable params: 608,704
Non-trainable params: 0
_________________________________________________________________


Load emnist lines and pass it to the new model

## Look at the data

In [104]:
df = pd.read_csv(Dataset.processed_data_path()/"emnist_lines"/"data.csv")

output_length = len(df)

In [105]:
df.head()

,image,sentence
0,0.png,Whats wrong at state _____________
1,1.png,The transducer itself moves the __
2,2.png,There is an ancient and venerable
3,3.png,They destroyed a trading house ___
4,4.png,Alec waited a moment on guard ___


In [106]:
image = Image.open(Dataset.processed_data_path()/"emnist_lines"/"0.png")
image

In [107]:
np.array(image).shape

(28, 952)

## Try it before training

In [108]:
image_numpy = np.array(image).astype(np.float32)
test_batch = image_numpy.reshape(1, image_height, image_width, 1)

preds = model(test_batch)
preds.shape

TensorShape([1, 67, 64])

In [109]:
image

In [110]:
for char_pred in preds[0]:
    print(mapping[np.argmax(char_pred)], end='')

rro0k55r0W00rr5rtrt5o500r5kf00rr6rKrt0KW000000000000000000000000000